In [8]:
import pandas as pd
import numpy as np
import csv
import math
from bs4 import BeautifulSoup
from gensim.summarization.bm25 import get_bm25_weights
from rank_bm25 import BM25Okapi
import json
from termcolor import colored
import re
import itertools
from functools import reduce

In [9]:
with open('./output_data/tmp/article_pool.json', 'r') as input_file:
    phrase_list = json.load(input_file)

data_df_layer_1 = pd.DataFrame(phrase_list[0])
layer_num = len(phrase_list)
unique_phrase_list = np.unique(data_df_layer_1['phrase'].values)

In [10]:
p_list = list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[0])

In [11]:
list_of_phrase_list = []
for i in range(len(unique_phrase_list)):
    tmplist =[]
    for j in range(len(p_list)):
        if p_list[j] in unique_phrase_list[i]:
            tmplist.append(p_list[j])
    list_of_phrase_list.append(tmplist)

In [12]:
def create_sentece_list(unique_phrase_list):
    list_of_phrase_list = []
    for i in range(len(unique_phrase_list)):
        tmplist =[]
        for j in range(len(p_list)):
            if p_list[j] in unique_phrase_list[i]:
                tmplist.append(p_list[j])
        list_of_phrase_list.append(tmplist)
    return list_of_phrase_list

In [13]:
#%%
#create the article pool now
article_list = data_df_layer_1['article'].values
#for each article, find all sentence
sentence_dic = {}
list_sentence = []
s_count = 0 #sentence index
for i in range(len(article_list)):
    #for every sentence, if not in sentence_list, push sentence in list
    tmp_sentence_list = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', article_list[i])
    for j in range(len(tmp_sentence_list)):
#         if tmp_sentence_list[j] not in article_list:
        sentence_dic[s_count] = tmp_sentence_list[j]
        list_sentence.append(tmp_sentence_list[j])        
        s_count +=1
list_sentence = np.unique(list_sentence)


In [14]:
p_list

['sulfite',
 'aromatic ring',
 'copper',
 'redox',
 'ATCC',
 'Desulfovibrio desulfuricans',
 'catalytic activity',
 'kinetics',
 'partially reduced',
 'photosystem II']

In [15]:
print(list(itertools.combinations(p_list,2)))

[('sulfite', 'aromatic ring'), ('sulfite', 'copper'), ('sulfite', 'redox'), ('sulfite', 'ATCC'), ('sulfite', 'Desulfovibrio desulfuricans'), ('sulfite', 'catalytic activity'), ('sulfite', 'kinetics'), ('sulfite', 'partially reduced'), ('sulfite', 'photosystem II'), ('aromatic ring', 'copper'), ('aromatic ring', 'redox'), ('aromatic ring', 'ATCC'), ('aromatic ring', 'Desulfovibrio desulfuricans'), ('aromatic ring', 'catalytic activity'), ('aromatic ring', 'kinetics'), ('aromatic ring', 'partially reduced'), ('aromatic ring', 'photosystem II'), ('copper', 'redox'), ('copper', 'ATCC'), ('copper', 'Desulfovibrio desulfuricans'), ('copper', 'catalytic activity'), ('copper', 'kinetics'), ('copper', 'partially reduced'), ('copper', 'photosystem II'), ('redox', 'ATCC'), ('redox', 'Desulfovibrio desulfuricans'), ('redox', 'catalytic activity'), ('redox', 'kinetics'), ('redox', 'partially reduced'), ('redox', 'photosystem II'), ('ATCC', 'Desulfovibrio desulfuricans'), ('ATCC', 'catalytic activit

In [16]:
a = list(itertools.combinations(p_list,3))

In [17]:
list(a[0])

['sulfite', 'aromatic ring', 'copper']

In [18]:
unique_phrase_list

array(['ATCC Desulfovibrio desulfuricans', 'ATCC catalytic activity',
       'ATCC kinetics', 'ATCC partially reduced', 'ATCC photosystem II',
       'Desulfovibrio desulfuricans catalytic activity',
       'Desulfovibrio desulfuricans kinetics',
       'Desulfovibrio desulfuricans partially reduced',
       'Desulfovibrio desulfuricans photosystem II', 'aromatic ring ATCC',
       'aromatic ring Desulfovibrio desulfuricans',
       'aromatic ring catalytic activity', 'aromatic ring copper',
       'aromatic ring kinetics', 'aromatic ring partially reduced',
       'aromatic ring photosystem II', 'aromatic ring redox',
       'catalytic activity kinetics',
       'catalytic activity partially reduced',
       'catalytic activity photosystem II', 'copper ATCC',
       'copper Desulfovibrio desulfuricans', 'copper catalytic activity',
       'copper kinetics', 'copper partially reduced',
       'copper photosystem II', 'copper redox',
       'kinetics partially reduced', 'kinetics photos

In [19]:
def ven_diagram(list_sentence,unique_phrase_list,p_list):
    #generate the list of list
    list_of_list_com = []
    counter = len(p_list)
    while counter > 0:
        list_of_list_com.append(list(itertools.combinations(p_list,counter)))
        counter-=1
        
    #venn cover
    len_p_list = len(p_list)#len of p_list
    #find the sentence that matches current cover
    ans = []
    for i in range(len(list_of_list_com)):
        for j in range(len(list_of_list_com[i])):
            #find the phrase list that not in com_list
            cur_list = list(list_of_list_com[i][j])
            not_in_list = []
            for pos in range(len(p_list)):
                if p_list[pos] not in cur_list:
                    not_in_list.append(p_list[pos])
            
            #find the sentence that cover all phrase in list_of_list_com[i][j] and not covered in not_in_list
            for p in  range(len(list_sentence)):
                flag_good = True
                flag_good_filter = True
                for q in range(len(cur_list)):
                    if cur_list[q] not in list_sentence[p]:
                        flag_good = False
                        break
                for g in range(len(not_in_list)):
                    if not_in_list[g] in list_sentence[p]:
                        flag_good_filter = False
                        break
                #pass all exam
                #push current setence to result
                if flag_good==True and flag_good_filter==True and list_sentence[p] not in ans:
                    ans.append(list_sentence[p])
                    
        print('combination:',len_p_list,' choose ',len_p_list - i,' current lengh of ans is', len(ans))
                    
    return ans

a = ven_diagram(list_sentence.copy(),unique_phrase_list,p_list)
len(a[1])

combination: 10  choose  10  current lengh of ans is 0
combination: 10  choose  9  current lengh of ans is 0
combination: 10  choose  8  current lengh of ans is 0
combination: 10  choose  7  current lengh of ans is 0
combination: 10  choose  6  current lengh of ans is 0
combination: 10  choose  5  current lengh of ans is 0
combination: 10  choose  4  current lengh of ans is 0
combination: 10  choose  3  current lengh of ans is 1
combination: 10  choose  2  current lengh of ans is 19
combination: 10  choose  1  current lengh of ans is 355


55

In [ ]:
#%%
def set_cover(sentence_list,unique_phrase_list,p_list):
    #at each iteration
        #find the sentence that cover most number of unvisted phrase
        #mark those phrase as visited (pop)
        #mark the sentence as visited (pop)
    answer_sentence_list = []
    touch_count_dic = []
    count = 0
    while len(p_list) > 0:
        #create a data structure to save how many unvisited phrase the current sentence touched
        touch_count_dic = {} #key as num of phrase touched, value is a list of index of sentence
        global_max_count = 0
        for i in range(len(sentence_list)):
            #compute num of touched
            tmpcount = 0
            for j in range(len(p_list)):
                if p_list[j] in sentence_list[i]:
                    tmpcount+=1
            if tmpcount > global_max_count:
                global_max_count = tmpcount
            # save current result in dic
            if tmpcount in touch_count_dic:
                #return the list
                curlist = touch_count_dic.get(tmpcount)
                curlist.append(i)
            else:
                tmplist = []
                tmplist.append(i)
                touch_count_dic[tmpcount] = tmplist
        #use the global max count to return lit of index of sentence that lead to the max current count
        
        list_of_max_index_sentence = touch_count_dic[global_max_count]
        #pick the first one
        selected_max_sentence_index = list_of_max_index_sentence[0]
        selected_max_sentence = sentence_list[selected_max_sentence_index]
        #set cover
        visited_list = []
#         print('what is sentence now', selected_max_sentence)
        
        for loc in range(len(p_list)):
            if p_list[loc] in selected_max_sentence and p_list[loc] not in visited_list:
                visited_list.append(p_list[loc])
        #delete all visited list
        print('!!!! visted list is', len(visited_list))
#         print('what is sentence now', selected_max_sentence)
        for pos2 in range(len(visited_list)):
            p_list.remove(visited_list[pos2])
        answer_sentence_list.append(selected_max_sentence)
        #remove the current sentencn
        sentence_list.pop(selected_max_sentence_index)
#         print('length of sentence list is', len(sentence_list))
#         print('len of remainng list', p_list)
    return answer_sentence_list

In [ ]:
#%%
def annotating_function(answer,p_list): #only mark the first occurance of a phrase exist in sentence
    #iter through every answer
    for i in range(len(answer)):
        for j in range(len(p_list)):
            if p_list[j] in answer[i]:
                #find the starting index
                start = answer[i].find(p_list[j])
                end = start + len(p_list[j])
                answer[i] = answer[i][0:start] + colored(p_list[j],'red') + answer[i][end:]
    for i in range(len(answer)):
        print('index :', i, '', answer[i] +'\n')


#%%
def main_func():
    with open('./output_data/tmp/article_pool.json', 'r') as input_file:
        phrase_list = json.load(input_file)
    layer_num = len(phrase_list)  #how many layer
    full_output = []
    for i in range(layer_num):
        data_df_layer_1 = pd.DataFrame(phrase_list[i])
        unique_phrase_list = np.unique(data_df_layer_1['phrase'].values)
        p_list = list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[i])
#         list_of_phrase_list =  create_sentece_list(unique_phrase_list)
        list_sentence = create_sentence_pool(data_df_layer_1)
        #run
        answer = ven_diagram(list(list_sentence),list(unique_phrase_list),p_list.copy())
        full_output.append(answer)          
        print('current layer is: ',i )
        annotating_function(answer.copy(),p_list)

    with open('./output_data/summaries.json', 'w') as outfile:
            json.dump(full_output, outfile, indent = 4)


#%%
# Runs the entire program
main_func()


